In [1]:
import matplotlib
matplotlib.use('Agg')
# Remove padding around screen

from IPython.core.display import display, HTML
import matplotlib.pyplot as plt
import matplotlib.image as img

display(HTML("<style>"
    + "#notebook { padding-top:0px !important; } " 
    + ".container { width:100% !important; } "
    + ".end_space { min-height:0px !important; } "
    + "</style>"))

In [2]:
from os import walk
import tensorflow as tf

import numpy
import keras
import PIL
from PIL import Image

output_angles = []
angle = -25
for i in range(45):
    output_angles.append(round(angle,2))
    angle+=1.155
    
def parse_img_name(fstr):
    data = fstr.split("_")
    if(len(data)>2):
        return(data[3])
    else:
        return -1

def prepare_files(path):
    # Numpy array containing images
    imgs = []
    # List containing steering angles
    steerings = []
    files = get_image_paths(path)
    # Prepare training data
    count = 0
    for fname in files:
        count+=1
        #if(count > 1000):
        #    break
        parsed = parse_img_name(fname)
        if not(parsed == -1): # Make sure the file is valid
            # steerings.append( parse_img_name(fname) ) # Append steering
            ohe_steering = one_hot_steering(parse_img_name(fname))
            if(ohe_steering == None):
                continue # Steering is not valid, continue without bad data
            steerings.append( ohe_steering )
            img = load_image(path+fname)    # Load the image
            img = numpy.array(img)          # Convert to numpy array
            B,G,R = channelSplit(img)
            G = G.flatten()
            imgs.append(G)                  # Append image to images array
    return imgs, steerings

def get_image_paths(path):
    # Walk through all the images in the folder path
    f = []
    for (dirpath, dirnames, filenames) in walk(path):
        f.extend(filenames)
    return f

def load_image(infilename):
    size = 32,30
    img = Image.open(infilename)
    img.thumbnail(size, Image.ANTIALIAS)
    data = numpy.asarray(img,dtype="uint8")
    return data

def channelSplit(image):
    return numpy.dsplit(image,image.shape[-1])

def one_hot_steering(angle):
    weights=[1,1,2,3,4,5,4,3,2,1,1]  # Weights to output feedback unit
    encoding = [0] * 45
    for i in range(len(output_angles)-1):
        if (output_angles[i] <= float(angle)) and (float(angle) <= output_angles[i+1]):
            for j in range(i-5,i+5,1): # Iterate through encoding and place weights
                if(j>=0 and j<(len(encoding))):
                    encoding[j] = weights[j+5-i]
            return encoding
        if (i == len(output_angles)-1):
            print("Data incorrectly formatted")
            return encoding
            break

Using TensorFlow backend.


In [3]:
filepath = "../datasets/dataset1/"
imgs, steerings = prepare_files(filepath)
train_y = numpy.array(steerings)
train_x = numpy.array(imgs)
# AC-frame_011922_st_23.87382_th_0.3.jpg
# one_hot_steering("24")
train_y.shape

(68509, 45)

In [ ]:
img = load_image("../datasets/testing/1242071frame_000026_st_0.4888471_th_0.3.jpg")
# Get only blue channel of image, set other
# channel intesnities to 0
img = numpy.array(img)
img[:, :, 1] = 0    # Zero out contribution from green
img[:, :, 2] = 0
imgplot = plt.savefig('sample2.jpg')
imgplot = plt.imshow(img)

In [4]:
train_x.shape[0]

68509

In [ ]:
def get_model(input_size):
    model = keras.models.Sequential()
    model.add(keras.layers.Dense(100, input_dim=input_size, activation='tanh'))
    model.add(keras.layers.Dropout(0.1))
    model.add(keras.layers.Dense(45,  activation='relu'))
    return model
# Get training image size
i = train_x.shape[1]

# Load callbacks
checkpointer = keras.callbacks.ModelCheckpoint(
    filepath='../outputs/ALVINNS1.h5',
    verbose=1,
    save_best_only=True)

model = get_model(input_size=i)

model.compile(optimizer='adam',
              loss='kullback_leibler_divergence',
              metrics=['accuracy'])

model.summary()

In [ ]:
model.fit(x=train_x,y=train_y,nb_epoch=10,validation_split=0.1,callbacks=[checkpointer])